# Ejecute esta celda para instalar las dependencias

In [ ]:
!pip install automata-lib==5.0.0

# *para ejecutar el codigo sin que se generen errores , es necesario subir la imagen de la cancha con el nombre 'a.jpg'*

si subio el notebook a google colab y la aimagen a su drive ejecute esta celda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
ruta_imagen = "./drive/MyDrive/a.jpg"

de otra forma, si la subio directamente al colab ejecute esta celda

In [ ]:
ruta_imagen = "./a.jpg"

# PROYECTO

EJECUTE TODAS LAS CELDAS DE CODIGO EN ORDEN 

la definicion formal se encuentra luego del codigo :)

In [3]:
from PIL import Image, ImageDraw, ImageFont
import random
import math
import numpy as np
from automata.fa.dfa import DFA  #cuando se encuentre en colaboratory
import re

image = Image.open(ruta_imagen)
draw = ImageDraw.Draw(image)
r = 14 #radio de 14
jugadores={}
condicion = {}
posicion_canasta = [569,169]

def dibujar_jugador(x, y, num,color):
    ListaDePuntos = (x-r, y-r, x+r, y+r)    
    draw.ellipse(ListaDePuntos, fill=color)
    font = ImageFont.load_default()
    draw.text((x-(r/2),y-(r/2)), "{:2}".format(num), font=font, fill=(255, 255, 255))
    jugadores[num] = (x, y)

def dibujar_equipo(color,min_number,max_number,rango_x,rango_y):
    for i in range(min_number,max_number):
        x,y = random.randint(rango_x[0], rango_x[1]) , random.randint(rango_y[0], rango_y[1])
        while any((x-r-12) < posicion[0] < (x+r+12) and (y-r-12) < posicion[1] < (y+r+12) for posicion in jugadores.values()): 
            x,y = random.randint(rango_x[0], rango_x[1]) , random.randint(rango_y[0], rango_y[1])
        dibujar_jugador(x, y, i,color)

def dibujar_jugador_inicial():
  jugador_inicial = min(equipo1,key=equipo1.get)
  dibujar_jugador(equipo1[jugador_inicial][0],equipo1[jugador_inicial][1],jugador_inicial,(0,0,0))
  return jugador_inicial

def distancia_entre_puntos(x1,y1,x2,y2):
  return math.sqrt( (x2 - x1)**2 + (y2 - y1)**2 )

def intersecta(p1, p2, c, r):
    c1 = np.subtract(c, p1)
    c2 = np.subtract(c, p2)
    dist1 = np.linalg.norm(c1)
    dist2 = np.linalg.norm(c2)
    if (r - dist1) * (r - dist2) < 0: return True
    if r > dist1: return False
    dp = np.subtract(p2, p1)
    t = dp.dot(c1) / dp.dot(dp)
    if t < 0 or t > 1: return False
    cp = np.subtract(p1 + t * dp, c)
    distp = np.linalg.norm(cp)
    return distp <= r

def jugador_mas_cercano(jugador1,ignorar):
  distancia_entre_jugadores = []
  for jugador,posicion in equipo1.items():
    if jugador != jugador1:
      distancia_entre_jugadores.append((jugador1,jugador,distancia_entre_puntos(jugadores[jugador1][0],jugadores[jugador1][1],posicion[0],posicion[1])))
  jugador_mas_cercano = min(distancia_entre_jugadores, key = lambda x: x[2])
  while jugador_mas_cercano[1] in ignorar.values() or jugador_mas_cercano[1] == jugador_inicial:
    distancia_entre_jugadores.remove(jugador_mas_cercano)
    jugador_mas_cercano = min(distancia_entre_jugadores, key = lambda x: x[2])
  return jugador_mas_cercano[1]

def jugadores_mas_cercanos():
  mas_cercanos = {}
  temp = jugador_inicial
  for i in range(4):
    mas_cercanos[temp] = jugador_mas_cercano(temp,mas_cercanos)
    temp = mas_cercanos[temp]
  return mas_cercanos , temp

def verificar_pase_entre_jugadores(jugador1,jugador2):
  obstaculos = []
  for pos in jugadores.values():
      if (pos[0] != jugadores[jugador1][0] and pos[1] != jugadores[jugador1][1]) and (pos[0] != jugadores[jugador2][0] and pos[1] != jugadores[jugador2][1]):
        result = (jugador1, jugador2 , intersecta([jugadores[jugador1][0],jugadores[jugador1][1]], [jugadores[jugador2][0],jugadores[jugador2][1]], [pos[0], pos[1]], r+2))
        obstaculos.append(result)
  pase = [(pase[0],pase[1],pase[2]) for pase in obstaculos if pase[2]]   
  return '1' if not pase else '0'

def verificar_distancia_entre_jugadores(jugador1,jugador2,equipo):
  return '1' if equipo[jugador2][0] >= equipo[jugador1][0] - 45 else '0'


In [4]:
def generar_palabras():
  #primera pregunta ¿se la puedo pasar?
  for jugador in mas_cercanos:
      condicion[mas_cercanos[jugador]] = verificar_pase_entre_jugadores(jugador,mas_cercanos[jugador])

  #segunda pregunta ¿tiene alguna opcion de pase?
  for jugador in condicion:
      if jugador == ultimo_jugador:
        condicion[jugador] = condicion[jugador] + '1'
      else:
        condicion[jugador] +=  verificar_pase_entre_jugadores(jugador,mas_cercanos[jugador])

  #tercera pregunta ¿esta a una distancia superior a 45 puntos detras del que la pasa?
  for jugador in mas_cercanos:
      condicion[mas_cercanos[jugador]] +=  verificar_distancia_entre_jugadores(jugador,mas_cercanos[jugador],equipo1)

  #cuarta pregunta ¿es el ultimo jugador?
  for jugador in condicion:
    condicion[jugador] += '1' if jugador == ultimo_jugador else '0'

  #quinta pregunta si es el ultimo jugador, ¿puede tirar?
  colisiones = []
  for pos in jugadores.values():
    if pos[0] != equipo1[ultimo_jugador][0] and pos[1] != equipo1[ultimo_jugador][1]:
      colisiones.append(intersecta([equipo1[ultimo_jugador][0],equipo1[ultimo_jugador][1]],posicion_canasta, [pos[0], pos[1]], r+2))
  colisiones = [tiro for tiro in colisiones if tiro]
  condicion[ultimo_jugador] = condicion[ultimo_jugador] + '1' if not colisiones else condicion[ultimo_jugador] + '0'

In [ ]:
def automata_filtrador():
  dfa1 = DFA(
      states={'q0', 'q1', 'q2', 'q3','q4','q5','q6','q7','q8','q9','q10'},
      input_symbols={'0','1'},
      transitions={
          'q0': {'0': 'q1', '1':'q2'},
          'q1': {'0': 'q1', '1':'q1'},
          'q2': {'0': 'q3', '1':'q4'},
          'q3': {'0': 'q3', '1':'q3'},
          'q4': {'0': 'q5', '1':'q6'},
          'q5': {'0': 'q5', '1':'q5'},
          'q6': {'0': 'q7', '1':'q8'},
          'q7': {},
          'q8': {'0': 'q9', '1':'q10'},
          'q9': {},
          'q10': {}
      },
      initial_state='q0',
      final_states={'q7','q10'},
      allow_partial = True 
  )
  return dfa1

dfa1 = automata_filtrador()

def automata_jugada(palabra):
  palabra = [caracter for caracter in palabra]
  dfa2 = DFA(
      states={'q0', 'q1', 'q2', 'q3','q4','q5','q6'},
      input_symbols={'0','1','2','3','4','5','t'},
      transitions={
          'q0': {palabra[0]: 'q1'},
          'q1': {palabra[1]: 'q2'},
          'q2': {palabra[2]: 'q3'},
          'q3': {palabra[3]: 'q4'},
          'q4': {palabra[4]: 'q5'},
          'q5': {palabra[5]: 'q6'},
          'q6': {},
      },
      initial_state='q0',
      final_states={'q6'},
      allow_partial = True 
  )
  return dfa2

def verificar_jugada():
  for jugador,palabra in condicion.items():
    try:
      estado_final = dfa1.read_input(palabra)
      if estado_final == 'q10':
        print('genereacion valida')
        return True
    except Exception as error:
      estado_final = 'q' + re.search(r'\d+', str(error)).group(0)
      if estado_final =='q1':
        print(f'generacion invalida, el jugador numero {jugador} no puede recibir el balon de {jugador_inicial}')
      elif estado_final == 'q3':
        print(f'generacion invalida, el jugador numero {jugador} no tiene opcion de pase')
      elif estado_final == 'q5':
        print(f'generacion invalida, el jugador numero {jugador} esta demasiado por detras de {list(mas_cercanos)[list(mas_cercanos.values()).index(jugador)]}')
      elif estado_final == 'q9':
        print(f'generacion invalida, el jugador numero {jugador} tiene un obstaculo y no puede tirar')
      else:
        print(error,jugador)
      generar_tablero()
      break

def generar_tablero():
  global image,draw,equipo1,equipo2,condicion, jugador_inicial, ultimo_jugador, mas_cercanos
  image.close()
  image = Image.open(ruta_imagen)
  draw = ImageDraw.Draw(image)
  jugadores.clear()
  condicion.clear()
  dibujar_equipo((0,0,255,),1,6,[306,560],[30,305]) #azul
  dibujar_equipo((255,0,0),6,11,[400,560],[70,280]) #rojo
  equipo1 = {jugador: posicion for jugador,posicion in jugadores.items() if jugador < 6}
  equipo2 = {jugador: posicion for jugador,posicion in jugadores.items() if jugador >= 6}
  jugador_inicial = dibujar_jugador_inicial()
  mas_cercanos, ultimo_jugador = jugadores_mas_cercanos()
  generar_palabras()
  verificar_jugada()

generar_tablero()

palabra = str(jugador_inicial) + ''.join(map(str, list(mas_cercanos.values()))) + 't'
automata_jugada = automata_jugada(palabra)

In [ ]:
display(image)

In [ ]:
if verificar_jugada():
  while True:
    try:
      palabra_jugador = input("\n -- escriba la jugada correcta --")
      estado_final = automata_jugada.read_input(palabra_jugador)
      print("\n **la jugada digitada es correcta**")
      break 
    except Exception as e:
      continue

# Definicion Formal









## automata_filtrador
sea automata_filtrador = {Q , ∑ , 𝛾 , q0 , F}

Q = {'q0', 'q1', 'q2', 'q3','q4','q5','q6','q7','q8','q9','q10'}

∑ = {'0','1'}

<details>
  <summary> 𝛾 (transiciones)</summary>

  𝛾( q0 , '0' ) = q1

  𝛾( q0 , '1' ) = q2

  𝛾( q1 , '0' ) = q1

  𝛾( q1 , '1' ) = q1

  𝛾( q2 , '0' ) = q3

  𝛾( q2 , '1' ) = q4

  𝛾( q3 , '0' ) = q3

  𝛾( q3 , '1' ) = q3

  𝛾( q4 , '0' ) = q5

  𝛾( q4 , '1' ) = q6

  𝛾( q5 , '0' ) = q5

  𝛾( q5 , '1' ) = q5

  𝛾( q6 , '0' ) = q7

  𝛾( q6 , '1' ) = q8

  𝛾( q8 , '0' ) = q9

  𝛾( q8 , '1' ) = q10
</details>

q0 = 'q0'

F = {'q7','q10'}

*diagrama:*

![imagen automata filtrador](https://i.imgur.com/nYHYFby.png)

## automata_jugada

sea automata_jugada = {Q , ∑ , 𝛾 , q0 , F}

Q = { 'q0' , 'q1' , 'q2' , 'q3' , 'q4' , 'q5' , 'q6' }

∑ = { '1', '2' , '3' , '4' , '5' , 't' }

<details>
  <summary> 𝛾 (transiciones)</summary>

  𝛾( q0 , pase[0] ) = q1

  𝛾( q1 , pase[1] ) = q2

  𝛾( q2 , pase[2] ) = q3

  𝛾( q3 , pase[3] ) = q4

  𝛾( q4 , pase[4] ) = q5

  𝛾( q5 , 't' ) = q6

</details>

q0 = 'q0'

F = {'q6'}

diagrama:

![diagrama automata jugada](https://i.imgur.com/qHMWisF.png)